In [2]:
import json
import pandas as pd
import ast
from datetime import datetime, time
from geopy.geocoders import Nominatim

En este archivo voy a juntar las dos tablas de hechos y accidentes ya limpias para dejar un solo archivo

In [ ]:
# Abro los archivos parquet que voy a juntar

hechos_original = r"C:\Users\Usuario\Henry\PI2_DA\ETLS y EDAS\df_hechos.parquet"
df_hechos = pd.read_parquet(hechos_original)
df_hechos

In [ ]:
victimas_original = r"C:\Users\Usuario\Henry\PI2_DA\ETLS y EDAS\df_victimas.parquet"
df_victimas = pd.read_parquet(victimas_original)
df_victimas

In [10]:
df_accidentes = pd.merge(df_hechos, df_victimas, on='ID_hecho')

In [ ]:
df_accidentes.head()

In [17]:
# Primero verifico los datos con la función

def verificar_tipo_datos(df):
# Genero Diccionario vació para almacenar información sobre cada columna
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}
    
# Iterar sobre  las columnas del DataFrame df
    for columna in df.columns:
        # Calcular el porcentaje de valores no nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        
# Agregar información de mi_dict
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())
        
# Crear un DataFrame con la información recopilada
    df_info = pd.DataFrame(mi_dict)
    
# Calcular el total de valores nulos en el DataFrame
    total_nulos = df.isnull().sum().sum()
    
# Imprimir el total de valores nulos
    print(f'Total de valores nulos en el DataFrame: {total_nulos}')
    
# Mostrar las columnas con valores nulos y sus respectivos porcentajes (ordenadas de mayor a menor)
    columnas_con_nulos = df_info[df_info["nulos"] > 0][["nombre_campo", "nulos_%"]].sort_values(by="nulos_%", ascending=False)
    if not columnas_con_nulos.empty:
        print("\nColumnas con valores nulos y sus porcentajes:")
        print(columnas_con_nulos)
        
# Devolver el DataFrame con la información de cada columna

    return df_info

In [18]:
verificar_tipo_datos(df_accidentes)

Total de valores nulos en el DataFrame: 0


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,ID_hecho,[<class 'str'>],100.0,0.0,0
1,N_VICTIMAS,[<class 'int'>],100.0,0.0,0
2,FECHA,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
3,HORA,[<class 'datetime.time'>],100.0,0.0,0
4,DIRECCION,[<class 'str'>],100.0,0.0,0
5,COMUNA,[<class 'int'>],100.0,0.0,0
6,VICTIMA_x,[<class 'str'>],100.0,0.0,0
7,ACUSADO,[<class 'str'>],100.0,0.0,0
8,COORDENADAS,[<class 'numpy.ndarray'>],100.0,0.0,0
9,ROL,[<class 'str'>],100.0,0.0,0


In [ ]:
# Super no hay datos nuclos. solo es necesario cambiar forma 

In [19]:
df_accidentes.size

9912

In [21]:
df_accidentes.head()

,ID_hecho,N_VICTIMAS,FECHA,HORA,DIRECCION,COMUNA,VICTIMA_x,ACUSADO,COORDENADAS,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,MOTO,AUTO,"[-58.47533969, -34.68757022]",CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,01:15:00,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,AUTO,PASAJEROS,"[-58.50877521, -34.66977709]",CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,07:00:00,ENTRE RIOS AV. 2034,1,MOTO,AUTO,"[-58.39040293, -34.63189362]",CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,00:00:00,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,MOTO,SD,"[-58.46503904, -34.68092974]",CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,1,2016-01-21,05:20:00,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,MOTO,PASAJEROS,"[-58.38718297, -34.62246630]",CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [22]:
# Cambio los siguientes nombres de columnas por presentacion:

df_accidentes = df_accidentes.rename(columns={'ID_hecho': 'ID_HECHO'})
df_accidentes = df_accidentes.rename(columns={'N_VICTIMAS': 'No_VICTIMAS'})
df_accidentes = df_accidentes.rename(columns={'FECHA': 'FECHA_HECHO'})
df_accidentes = df_accidentes.rename(columns={'HORA': 'HORA_HECHO'})
df_accidentes = df_accidentes.rename(columns={'SEXO': 'SEXO_VICTIMA'})    
df_accidentes = df_accidentes.rename(columns={'EDAD': 'EDAD_VICTIMA'})  
    

In [ ]:
df_accidentes.head()

In [26]:
# Borro columna victima por que está repetida

df_accidentes = df_accidentes.drop('VICTIMA_y', axis=1)


In [ ]:
df_accidentes.head()

In [29]:
# Guardo Df para hacer popsteriormente el EDA

df_accidentes.to_parquet('df_accidentes.parquet')
